In [1]:
import shutil
import numpy as np
import os
import argparse

In [2]:
def get_files_from_folder(path):
    files = os.listdir(path)
    return np.asarray(files)


In [4]:
get_files_from_folder("../Data/Plant_without_augmentation/Apple___Apple_scab")

array(['image (326).JPG', 'image (263).JPG', 'image (371).JPG',
       'image (234).JPG', 'image (408).JPG', 'image (367).JPG',
       'image (388).JPG', 'image (222).JPG', 'image (330).JPG',
       'image (275).JPG', 'image (625).JPG', 'image (449).JPG',
       'image (432).JPG', 'image (598).JPG', 'image (71).JPG',
       'image (577).JPG', 'image (127).JPG', 'image (609).JPG',
       'image (259).JPG', 'image (26).JPG', 'image (465).JPG',
       'image (170).JPG', 'image (520).JPG', 'image (5).JPG',
       'image (473).JPG', 'image (189).JPG', 'image (30).JPG',
       'image (536).JPG', 'image (166).JPG', 'image (218).JPG',
       'image (67).JPG', 'image (424).JPG', 'image (131).JPG',
       'image (88).JPG', 'image (561).JPG', 'image (516).JPG',
       'image (146).JPG', 'image (453).JPG', 'image (10).JPG',
       'image (280).JPG', 'image (111).JPG', 'image (541).JPG',
       'image (47).JPG', 'image (404).JPG', 'image (392).JPG',
       'image (238).JPG', 'image (557).JPG', 'ima

In [5]:
def parse_args():
    parser = argparse.ArgumentParser(description="Dataset divider")
    parser.add_argument("--source", required=True, 
                        help="path to source file")
    parser.add_argument("--target", required=True, 
                        help="path to save output file")
    parser.add_argument("--ratio", required=True, 
                        help="Train ratio, E.g. 0.7 means splitting data into 70% training and 30% testting")
    return parser.parse_args()

In [ ]:
def main(source, target, train_ratio):
    #get dirs
    _, dirs, _ = next(os.walk(source))

In [20]:
next(os.walk("../Data/Plant_without_augmentation/Apple/"))[:-1]

('../Data/Plant_without_augmentation/Apple/',
 ['Apple___Cedar_apple_rust',
  'Apple___Apple_scab',
  'Apple___Black_rot',
  'Apple___healthy'])

In [8]:
for i in os.walk("../Data/"):
    print(i)

('../Data/', ['Plant_without_augmentation', 'Plant_with_augmentation'], ['.DS_Store'])
('../Data/Plant_without_augmentation', ['Tomato___Target_Spot', 'Tomato___Late_blight', 'Tomato___Tomato_mosaic_virus', 'Tomato___Leaf_Mold', 'Apple___Cedar_apple_rust', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Apple___Apple_scab', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Background_without_leaves', 'Tomato___Septoria_leaf_spot', 'Apple___Black_rot', 'Apple___healthy'], ['.DS_Store'])
('../Data/Plant_without_augmentation/Tomato___Target_Spot', [], ['image (776).JPG', 'image (1151).JPG', 'image (326).JPG', 'image (633).JPG', 'image (1014).JPG', 'image (263).JPG', 'image (799).JPG', 'image (1297).JPG', 'image (849).JPG', 'image (1278).JPG', 'image (1106).JPG', 'image (371).JPG', 'image (721).JPG', 'image (1043).JPG', 'image (234).JPG', 'image (664).JPG', 'image (408).JPG', 'image (1385).JPG', 'image (737).JPG', 'image (3